<a href="https://colab.research.google.com/github/hungry2know/genai/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing environment variables for google cloud and Neo4J cloud instance


In [ ]:
import os

from google.colab import userdata
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_CLOUD_LOCATION"]=userdata.get('GOOGLE_CLOUD_LOCATION')
os.environ["GOOGLE_CLOUD_PROJECT"]=userdata.get('GOOGLE_CLOUD_PROJECT')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]=userdata.get('GOOGLE_GENAI_USE_VERTEXAI')
os.environ["NEO4J_PASSWORD"]=userdata.get('NEO4J_PASSWORD')
os.environ["NEO4J_URI"]=userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"]=userdata.get('NEO4J_USERNAME')

# Installing dependencies

In [ ]:
%pip install --upgrade langchain langchain-community langchain-google-genai langchain-google-vertexai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

# Google Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project {os.environ["GOOGLE_CLOUD_PROJECT"]}
!gcloud auth application-default set-quota-project {os.environ["GOOGLE_CLOUD_PROJECT"]}

# Loading ANZ (bank) article from wikipedia in Neo4j graph

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="ANZ (bank)").load()

from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.5-flash")

from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

[]

# Retrieval

In [ ]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project {os.environ["GOOGLE_CLOUD_PROJECT"]}
!gcloud auth application-default set-quota-project {os.environ["GOOGLE_CLOUD_PROJECT"]}

from langchain_community.vectorstores import Neo4jVector

from langchain_google_vertexai import VertexAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    VertexAIEmbeddings(model_name="text-embedding-005"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Updated property [core/project].

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "learn-langchain-466102" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

entity_chain.invoke({"question": "Who is latest CEO of ANZ bank?"}).names

['ANZ bank']

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    print(f"structured_data : {structured_data}")
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data


_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer



_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGoogleGenerativeAI(temperature=0, model="gemini-2.5-flash")
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

prompt = ChatPromptTemplate.from_template(template)


chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "Who is latest CEO of ANZ bank?"})

Search query: Who is latest CEO of ANZ bank?


structured_data : Launceston - LOCATED_IN -> Van Diemens Land
Anz Bank - HEADQUARTERED_IN -> Melbourne
Anz Bank - LOCATED_IN -> Australia
Anz Bank - IS_A_TYPE_OF -> Major Bank
Anz Bank - OPERATES_IN -> Australia
Anz Bank - OPERATES_IN -> New Zealand
Anz Bank - FORMED_BY_MERGER_OF -> English, Scottish & Australian Bank
Anz Bank - FORMED_BY_MERGER_OF -> Australia And New Zealand Bank
Anz Bank - IS_MEMBER_OF -> Big Four Australian Banks
Anz Bank - LARGEST_BANK_IN -> New Zealand
Anz Bank - OPERATED_AS -> Anz National Bank
Anz Bank - OPERATED_AS -> Anz Bank New Zealand
Anz Bank - OPERATED_BRAND -> National Bank Of New Zealand
Anz Bank - ACQUIRED -> National Bank Of New Zealand
Anz Bank - ACQUIRED -> Postbank
Anz Bank - ACQUIRED -> Ing New Zealand Limited
Anz Bank - ESTABLISHED_FACILITY -> Data Processing Centre
Anz Bank - BEGAN_OPERATIONS_IN -> Honiara
Anz Bank - OPENED_OFFICE_IN -> New York
Anz Bank - ESTABLISHED_OFFICE_IN -> Tokyo
Anz Bank - MERGED_WITH -> English, Scottish & Australian B

'Mike Smith.'